# Computation of climate variables

In the following we compute the variables from the fetched weather data.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from tqdm import tqdm 

In [2]:
# path of weather db
path = "prod/weather.db"
path_climate = "prod/climate.db"

In [3]:
# create sql engine
engine = create_engine("sqlite:///"+path, echo=False)
engine_climate = create_engine("sqlite:///"+path_climate, echo=False)
# get all table names
insp = inspect(engine)
tables = insp.get_table_names()

In [4]:
# get all tables with weather data
tables = list(filter(
    lambda x: "weather" in x,
    tables
))

In [5]:
# function computing vars
def compute_vars(data):
    # compute monthly averages
    return data.rolling("30D", min_periods=20).mean()

In [6]:
# read all the data and save as csv
for table in tqdm(tables, total=len(tables)):
    raw = pd.read_sql_table(
        table, 
        engine,
        parse_dates=True
    ).set_index('time')
    # create new table name
    index = table.replace("weather", "climate")
    # split table
    numeric = raw["tavg tmin tmax prcp snow wdir wspd wpgt pres tsun".split()]
    other = raw["nuts_name sloc".split()]
    # compute monthly averave
    numeric = compute_vars(numeric)
    # merge and save
    numeric.join(other).to_sql(index, con=engine_climate, if_exists="replace")

100%|███████████████████████████████████████████| 16/16 [00:36<00:00,  2.26s/it]
